In [1]:
import numpy
import os
import numpy
import re
from nltk.corpus import stopwords
import math
import json

In [2]:
files=open('sept19_26.json').readlines()
docs=[json.loads(file)["text"] for file in files]

In [3]:
def getbestCandidate(terms,term2docs,database):
    eos=[]
    selectedterm=frozenset({})
    selecetdeoss=999999999999
    for i in database:
        try:
            eos.append(1/len(i))
        except:
            eos.append(0)
    for term in terms:
        eossum=0
        for doc in term2docs[term]:
            eossum+=eos[doc]
        if eossum<selecetdeoss:
            print(eossum)
            selectedterm=term
            selecetdeoss=eossum
    print("TEEERRRMM: "+str(selectedterm))
    return selectedterm
        
def knowminsup(term,docs,database,ind2doc,minsup):
    termdocs=[]
    cond=False
    for i in list(docs):
        if term.issubset(ind2doc[i]):
            termdocs.append(i)
    if(len(termdocs)>=len(database)*minsup):
        cond=True
        for i in list(docs):
            database[i]=database[i].union(term)
            #print("new Term: "+str(term))
    return docs,database,cond
def cover(testSet,term2docs):
    return(len(term2docs[testSet]))
def candidate_gen(database,terms,term2docs,ind2doc,minsup):
    k = 1
    lterms=list(terms)
    print(len(lterms))
    while(True):

        newterms=[]
        print(len(lterms))
        if(len(lterms)==0):
            break
        for i in range(len(lterms)):
            for j in range(i + 1, len(lterms)):
                term = lterms[i].union(lterms[j])
                if len(term) == k: 
                    #print("passed")
                    continue
                docs=term2docs[lterms[i]].union(term2docs[lterms[j]])
                docs,database,cond=knowminsup(term,docs,database,ind2doc,minsup)
                if (cond):
                    #print("term: "+str(term))
                    #print("docs: "+str(docs))
                    term2docs[term]=docs
                    newterms.append(term)
        print("new terms: "+str(frozenset(newterms)))
        terms=terms.union(frozenset(newterms))
        lterms=list(frozenset(newterms))
        k+=1
    print("len total terms: "+str(len(terms)))    
    return database,terms,term2docs
def preproc(database,minsup): 
    database=[document.lower() for document in database]
    database=[re.sub("[\!\.\-\,\?(\n)\:\¿\¡\(\)\[\]]*","", document) for document in database]
    stp=stopwords.words('spanish')
    new_database=[]
    for document in database:
        new_database.append(" ".join([word for word in document.split(" ")\
                                      if (word not in stp)]))
    totalToks=(" ".join(new_database)).split(" ")
    tipos=list(set(totalToks))
    print("total de tokens :"+str(len(totalToks)))
    print("total de tipos :"+str(len(tipos)))
    w2i={}
    i2w={}
    for i,word in enumerate(tipos):
        w2i[word]=i
        i2w[i]=word
    database=[]
    for document in new_database:
        database.append(frozenset(frozenset([w2i[word]]) for word in document.split(" ")))
    ind2doc={}
    doc2ind={}
    for i,doc in enumerate(database):
        ind2doc[i]=(set(w2i[word] for word in new_database[i].split(" ")))
        doc2ind[doc]=i
    term2docs={}
    newterm2docs={}
    for document in database:
        for term in document:
            if(term in term2docs.keys()):
                term2docs[term]=term2docs[term].union({doc2ind[document]})
            else:
                term2docs[term]={doc2ind[document]}
    terms=term2docs.keys()
    
    #check minsup
    len(database)*minsup
    for term in terms:
        if (len(term2docs[term])>len(database)*minsup):
            newterm2docs[term]=term2docs[term]
        else:
            pass
            #print("poped "+str(term))     
    terms=frozenset(newterm2docs.keys())
    newdatabase=[]
    for i,document in enumerate(database):
        doc=database[i].intersection(terms)
        if len(doc)>0:
            newdatabase.append(document)
        else:
            newdatabase.append(0)
            #print("poped doc :"+str(document))
    return newdatabase,w2i,i2w,terms,newterm2docs,doc2ind,ind2doc,term2docs

In [4]:
def FTC(database, minsup):
    database,w2i,i2w,terms,term2docs,doc2ind,ind2doc,bla=preproc(database,minsup)
    ###Starting real FTC
    database,terms,term2docs=candidate_gen(database,terms,term2docs,ind2doc,minsup)
    totalcov=0
    selectedterms=[]
    aux=0
    while(totalcov!=len(database)):
        term=getbestCandidate(terms,term2docs,database)
        if term==frozenset():
            break
        #upgrade totalcov
        totalcov+=len(term2docs[term])
        #delete documents and terms
        newterms=[]
        for olterm in terms:
            if len(olterm.union(term))==(len(olterm)+len(term)):
                newterms.append(olterm)
                term2docs[olterm]=term2docs[olterm].difference(term2docs[term])
        terms=frozenset(newterms)
        for i in term2docs[term]:
            database[i]=0
        #add term to selected terms
        selectedterms.append(term)
    return selectedterms,term2docs,i2w,totalcov
selectedterms,term2docs,i2w,totalcov=FTC(docs,0.1)

total de tokens :388311
total de tipos :70778
5
5
new terms: frozenset()
0
len total terms: 5
464.91463878827574
TEEERRRMM: frozenset({56684})
997.0782243384656
969.4728230232461
517.60588700944
TEEERRRMM: frozenset({0})
853.7974578483343
TEEERRRMM: frozenset({38111})
1098.1535894119706
853.7128518884048
TEEERRRMM: frozenset({49088})
959.1295298930673
TEEERRRMM: frozenset({11276})
TEEERRRMM: frozenset()


In [5]:
print(totalcov)

27955


In [6]:
jsons=[json.loads(file) for file in files]
finalSons=[]
for term in selectedterms:
    sons=[]
    
    print("CLUSTER!!")
    print(len(term2docs[term]))
    for i in term2docs[term]:
         print(docs[i])
         sons.append(jsons[i])
    
    maysave=input()
    if maysave=="y":
        print("json saved!!")
        finalSons=finalSons+sons

CLUSTER!!
4520
18:12 SE ENTREGARON MÁS GUANTES EN MEDELLÍN Y EN ORIZABA, AHORA VAN A UN CENTRO DE ACOPIO EN COYOACÁN A ENTREGAR EL RESTO
Tenemos centro de acopio en ayuda a los damnificados por el sismo, personal de #SSPCDMX se instala en la Glorieta d… https://t.co/e5mzcsBjpK
La @andaestadiorg es un centro de acopio para apoyar a la población de Oaxaca y Chiapas afectada por el sismo. 
https://t.co/Ypc5G3nys2
Se necesitan relevos en el centro de acopio del estadio Universitario a las 7am.
Además del acopio, la @UNAM_MX apoya con ayuda psicológica para víctimas

#MéxicoUnido #UNAM #FuerzaMéxico #Sismo https://t.co/JxgvGeFX37
@lopezdoriga El Centro de acopio de la delegación Benito Juárez está llenísimo! Que manden por provisiones!!Comida, agua, material de todo!
Personal de la #SSPCDMX está recibiendo su apoyo para los damnificados del sismo, en el centro de acopio instalado… https://t.co/ZOy0Rl2s8Q
Personal de la #SSPCDMX está recibiendo su apoyo para los damnificados del sismo, en el

Nos unimos como centro de acopio para nuestros hermanos te esperamos en Divol Lindavista Av. Insurgentes Norte No.… https://t.co/xAMJKl0Lq7
Estos centros de acopio estan muy bien coordinados, ya se llevaron nuestras casas de campana y kits de higiene a Mo… https://t.co/iqhZyu7MP8
Q bonito, salir a la calle y ver en patios d escuelas, sitios d taxis, negocios, esquinas, centros de acopio o haciendo tortas #FueraMexico
Los espero en Bahía de Santa Barbara 119.
Centro de acopio de herramientas.

!COMPARTAN! https://t.co/cBJMpzu7VV
@AristeguiOnline En el centro de acopio de la fuente los Cibeles ya no hay víveres, hace falta medicamento alimentos enlatados, Etc.. RT
Centros de acopio que necesitan apoyo!!! https://t.co/T2Hd7JiiXX
Urgen sillas de ruedas, para centro de acopio en Berlín 107 Col Del Carmen, Coyoacán (22/09 - 10am)
@JaccPage Por supuesto. Hay centros de acopio saturados y sin transporte. Lo que puedan acercar acá para distribuir… https://t.co/OmJ1fWlulK
Centro de acopio, voy a

y
json saved!!
CLUSTER!!
5008
Thank you so much @ShawnMendes  #FuerzaMexico #PrayForMexico #PrayForMexico #PrayersforMexico #sismocdmx… https://t.co/I0zyUAiIRm
Mapa con información verificada de afectaciones por el sismo - Chilango https://t.co/fYYcS9jF03
@UNAM_MX @PumasMX Si necesitan transporte para víveres, material de curación, personal  o herramientas, tengo a su disposición mi camioneta
Busquen su albergue más cercano ahí también necesitan víveres!  #FuerzaMexico
Es muy bonito ver como la mayoría de mis amiguitos y un chingo de gente está ayudando #FuerzaMexico  #mexicoestadepie
Cu hay mucha ayuda (@ Estadio Olímpico Universitario - @estadioou in Coyoacán, CDMX) https://t.co/7ANsh43uL5
Ayuda a difundir, puede ser de utilidad
#FuerzaMexico #AyudaCDMX 
#mexicoestadepie #MexicoUnido  #TerremotoEnMexico… https://t.co/RCLjAs9RsL
Dra. Elena Orozco pide ayuda por celular está atrapada en la calle de Medellín 153 col. Roma . Urge!!!
#AyudaCiudadana
@SkyAlertMx  hubo un terremoto de 5.8 y

#copiamiapuesta sólo publicáremos  información de ayuda que a través de uds puede salvar 1 vida. Si quieres picks solicítalo vía DM
Comentan que en obrero mundial necesitan ayuda porque acaba de colapsar edificio, confirmen por favor!  #AyudaSismo#ayudaCDMX
@AztecaNoticias  por favor promueban la ayuda al Estado de Mexico y Morelos hay comunidades en situaciones precarias y no hay ayuda
URGE Recogiendo #AlvaroObregon286 #NoPodemosDejarlosAhi 15min p/que terminen72hr @ManceraMiguelMX @EPN  despidieron ayuda #Japon @JannTM RT
Usen esta app si se les extravío su mascota 🐶🐱 https://t.co/FZo6LKi25z  #sismo #Sismomexico @En_laDelValle… https://t.co/bKK8T8C1RH
@RicardoMonrealA @D_CuauhtemocDF @ManceraMiguelMX  revisión en amsterdam y parras en la Roma, por favor piden vecinos ayuda.
19 de Septiembre de 2017, sismo de  7.1 grados azota la gran ciudad.
La tristeza nos invade, la desesperación, el miedo... #TodosSomosMexico
No hay que olvidar que nuestra clase politica se les exigió su cooperaci

y
json saved!!
CLUSTER!!
7229
@LicAlbarran Cada ser humano nos ayuda a crecer hasta eso debemos agradecer 🤗
Está bien... no me quieres cerca, no me necesitas ahora, pero no olvides que estoy aquí cuando nuevamente requieras de mi ayuda.
Ayuda a compartir por favor. https://t.co/VXZ9TN2lTg
BUENAS NOCHES!!!
La desilución no es más que una situación, que te ayuda a salir del lugar… https://t.co/XoAxE4scH7 https://t.co/SEV8NnpFYJ
BUENAS NOCHES!!!
La desilución no es más que una situación, que te ayuda a salir del lugar… https://t.co/9UW3SrT2Pi… https://t.co/ghi4cmbdsi
Ayuda porfavor. @RinBosque @PolancoLomas @ComiteVPolanco @LibanoParque @AnzuresCDMX @VecinosvAnzures @saburtomh gra… https://t.co/T802uRezG3
@SkyAlertMx Y porque no cuentan que el gobierno no hizo nada y no acepto ayuda internacional.
⚠ Solicitud de ayuda para brigada.
#AyudaCDMX #FuerzaMéxico #MéxicoUnido #Relevos https://t.co/WcLMEX0WPY
Recuerdo que al casi amor de mi vida le pedí ayuda en una tarea sobre el SNA y aún no me

...

n
CLUSTER!!
5666
Buenos días!Recuerden hoy 11am se activará #AlertaSismica como señal de recuerdo del #sismo que literalmente sacudi… https://t.co/QgMS5iahWC
Filtran más videos de rescatistas #SISMO #CDMX https://t.co/YO1yrEsrw9 @LorenzoOkie @alynmon @MalthusGamba @san_yorch @PatFlova @Angelik9091
Hace 32 años. Estaba en casa de mis abuelitos cuando nos sorprendió ese gran #sismo
#Sismo85 | "Ya comenzó misa con sobrevivientes y personas que apoyaron durante #sismo del 85 en Tlatelolco":… https://t.co/N3UTxIm7mk
#Sismo | SEP estimó que necesitan 2 mil MDP para arreglar/reconstruir 2 mil 678 escuelas afectadas en Chiapas y Oax… https://t.co/MbEvu1bre4
@EPN recuerda el sismo de 1985, reitera el apoyo en reconstrucción al #sismo del pasado 7 de septiembre y resalta l… https://t.co/bLQFY8vSyx
#socios hoy es día de simulacros de #sismo en #CDMX lo molesto no es el simulacro lo molesto es no participar
😬😬los están guardando o etiquetando como si el gob lo donara tacha código de barras y escri

#monitoreodevivienda #sismo manden fotos para asesoria. https://t.co/1iHbq1FFc4
Hoy fui testigo de mi mejor México. Descansen para relevar a los que trabajaron hoy, que este día no terminará pronto. #Sismo #FuerzaMexico
Apoyemos dentro de nuestras posibilidades. #Sismo #FuerzaMexico #Mexicanosfuertes https://t.co/39IEOPign5
Por que cantando se alegra,cielito lindo los corazones #VivaMexico #EstoEsMiPais #Sismo #FuerzaMexico #SismoMx… https://t.co/wCEotlCTef
Este fue un día extremadamente triste. #Sismo
#Sismo #AyudaCiudadana https://t.co/lWwKBt5eZe
#Sismo #TenemosSismo #FuerzaMexico #fuerzaCdMx https://t.co/ggNUnW9IeG
Cansado, dolorido, pero orgulloso de ver de qué estamos hechos los mexicanos. Gracias al Ejercito militar y al ciudadano. #FuerzaCDMX #Sismo
Debido al #sismo se reportan ya 226 fallecidos: 55 en #Morelos, 117 en #CDMX, #39 en #Puebla, 12 en #Edoméx y 3 en #Guerrero.
#LutoNacional
Si perdiste o rescataste un animal de compañía en Ciudad de #México menciona a @AnimaNat_Mexi

n
CLUSTER!!
5532
La sociedad civil una vez más demuestra su apoyo en situaciones de desastre #FuerzaMexico https://t.co/koI1tBHvVQ
Conmovedora imágen #Sismomexico2017 #FuerzaMexico https://t.co/pGtnPrkuVR
#FuerzaMexico #pokemon torneo para juntar fondos en el centro https://t.co/dpMVGowzd3
#FuerzaMexico #pokemon los movimientos para recuperarnos del #TerremotoEnMexico https://t.co/LTKqRvLpqk
Hablé a par de hospitales en cdmx y recibiran donadores de sangre hasta el lunes, si alguien ubica donde p/hoy avise para ir #FuerzaMexico
Estas son gestas heroicas #FuerzaMexico https://t.co/gTku4zscnA
Me rompí amigos. Ya no pude más y decidí parar por hoy. Gracias a todos los que siguen dando el 110% #FuerzaMexico
Quienes trabajamos desde, para y con las juventudes nunca hemos dudado de lo que son capaces! #FuerzaMexico https://t.co/A4Y2zSw8eh
#FuerzaMexico vámonos a Morelos
Disfrutemos de la vida, valoremos que estamos aquí, saldremos adelante porque somos grandes y unidos #FuerzaMexico #FelizSa

#FuerzaMexico Te vas a levantar. https://t.co/r7bqgHO5t8
#IMPORTANTE | Para los que salgan a apoyar #FuerzaMexico #MexicoUnido https://t.co/TGcT3P7uux
@Benzema 🙏🙏🙏🙏 #FuerzaMexico 🇲🇽
#FuerzaMexico #CDMX https://t.co/6v4chygAT8
#toposmexico #CruzRojaMexicana 
Opciones excelentes para ayudar a México #FuerzaMexico 
Great ways to help Mexico… https://t.co/OdSSfwzrkn
Siempre México 🇲🇽❤ #FuerzaMexico https://t.co/NbQ1EvxQnY
Ayudar a los demás es como ayudarse a sí mismo. #FuerzaMexico https://t.co/7QDYesPKnv
#FuerzaMexico exceso de voluntarios entorpece labores de rescate https://t.co/3uYbmBbdID
@ElRorroGlz Hay que llevar las provisiones separadas y seleccionadas gracias por el apoyo #cdmx #FuerzaMexico por f… https://t.co/7gR4uPym3I
#AyudaCDMX #SismoCDMX #FuerzaMexico 🇲🇽️ https://t.co/AYt1NG2ElB
Las formas de ayudar son infinitas apoya como puedas #AyudemosMexico #ayudaCDMX #FuerzaMexico #Sismomexico #sismo… https://t.co/Hdom4oPRYC
#fuerzamexico #terremoto 
Unidos podemos todos. Ayudando. H

Ángeles de 4 patas #FuerzaMexico https://t.co/1RKXsqAzHE
De todas las formas se puede ayudar. 

#FuerzaMexico #MexicoUnido #AquiSeNecesita #MexicoEstaDePie https://t.co/MiyDv9Gath
Sin distinción de género Las chicas apoyando en el rescate #YoXMexico 👷🏼‍♀️ #MexicoEstaDePie 🔦⛏ #FuerzaMexico 🙏🇲🇽 @daniellemx_ @lopezdoriga
Sin distinción de género Las chicas también en el rescate #YoXMexico 👷🏼‍♀️ #MexicoEstaDePie ⛑🔦⚒⛏ #FuerzaMexico 🙏🇲🇽… https://t.co/z516iiji2E
#Video #YoSoyMarina #LaMarinaCercadeTi #FuerzaMexico https://t.co/6NBdHVKxry
#BuenasNoches #Vecino Recuerda que #Mexico sigue vivo! #FuerzaMexico #BLuna https://t.co/p1s6QLgbKn
Espero encuentren paz en sus corazones muy pronto. Animo. #fuerzamexico 🇲🇽❤️
No podría estar mad orgullosa de ser mexicana 🇲🇽 

#FuerzaMexico
Se necesita en Alvaro Obregon 286 
@makonancarrow #FuerzaMexico https://t.co/oLuYAPTs8F
#FuerzaMexico esto es la Marina Armada 👏🏻👏🏻👏🏻 https://t.co/DH2H9c1nHh
#FuerzaMexico #Marina #EjércitoNacional salvan vidas! https://t

In [7]:
len(finalSons)

9528

In [8]:
len(jsons)

40665

In [10]:
finalJ=json.dumps(finalSons)

In [15]:
with open('cluster.json', 'w+') as outfile:
    json.dump(finalJ, outfile)